In [1]:
import time
start1 = time.time()

In [2]:
from nbconvert.preprocessors import ExecutePreprocessor
import nbformat

# ✅ Flag de control
ejecutar_ingestion = False  # ← cambialo a True si querés ejecutar la ingesta

if ejecutar_ingestion:
    ruta_ingesta = "D:/trading/notebooks/sandbox/test_ingestion_Twelve_Data_API.ipynb"
    with open(ruta_ingesta, encoding="utf-8") as f:
        nb = nbformat.read(f, as_version=4)

    print("🚀 Ejecutando notebook de ingesta de datos...")
    ep = ExecutePreprocessor(timeout=600, kernel_name="python3")
    ep.preprocess(nb, {"metadata": {"path": "D:/trading/notebooks/sandbox"}})
    print("✅ Ingesta finalizada.")
else:
    print("⏭️ Ingesta omitida. Continuando con el proceso...")


⏭️ Ingesta omitida. Continuando con el proceso...


In [3]:
# ✅ Paso 2: Importar dependencias
import os
import pandas as pd
from datetime import datetime
from pathlib import Path
from importlib import import_module
import sys

In [4]:
# ✅ Paso 3: Configuracion de rutas
BASE_DIR = Path("D:/trading")
HISTORIC_DIR = BASE_DIR / "data/historic_reciente"
ESTRATEGIAS_PATH = BASE_DIR / "my_modules/estrategias"
OUTPUT_PATH = BASE_DIR / "reports/senales_heuristicas/diarias"
CONFIG_PATH = BASE_DIR / "config/450/symbol_groups.json"

In [5]:
# ✅ Paso 4: Cargar estrategias tuneadas
desarrollos = {}
sys.path.append(str(BASE_DIR))  # para que my_modules sea importable

for archivo in os.listdir(ESTRATEGIAS_PATH):
    if archivo.endswith(".py"):
        nombre_modulo = archivo[:-3]
        try:
            mod = import_module(f"my_modules.estrategias.{nombre_modulo}")
            desarrollos[nombre_modulo] = mod.generar_senales
            print(f"✅ Estrategia cargada: {nombre_modulo}")
        except Exception as e:
            print(f"❌ No se pudo cargar {nombre_modulo}: {e}")

✅ Estrategia cargada: bollinger_breakout_v4
✅ Estrategia cargada: cruce_medias_v4
✅ Estrategia cargada: gap_open_strategy_v5
✅ Estrategia cargada: ruptura_volumen_v1


In [6]:
import json
from pathlib import Path

# === CONFIGURACIÓN ===
CARGAR_TODOS_HISTORICOS = True  # ← cambia a False si querés usar el archivo json
HISTORIC_DIR = Path("D:/trading/data/historic")
CONFIG_PATH = Path("D:/trading/config/symbol_groups_prior.json")

# === CARGA DE SÍMBOLOS ===
if CARGAR_TODOS_HISTORICOS:
    simbolos = sorted([f.stem for f in HISTORIC_DIR.glob("*.parquet")])
    print(f"✅ Modo: todos los históricos ({len(simbolos)} símbolos)")
else:
    with open(CONFIG_PATH, "r", encoding="utf-8") as f:
        grupos = json.load(f)
    simbolos = sorted(set(sum(grupos.values(), [])))
    print(f"📂 Modo: grupo configurado ({len(simbolos)} símbolos)")


✅ Modo: todos los históricos (429 símbolos)


In [7]:
import time
from tqdm import tqdm
from joblib import Parallel, delayed
from datetime import date

# Crear salida si no existe
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f"Borrando señales anteriores en {OUTPUT_PATH}...")
for archivo in OUTPUT_PATH.glob("*.csv"):
    archivo.unlink()
print("✅ Directorio limpio.")

# === FUNCIÓN PARALELA POR SÍMBOLO ===
def procesar_simbolo(simbolo):
    archivo = HISTORIC_DIR / f"{simbolo}.parquet"
    if not archivo.exists():
        return f"⚠️ {simbolo}: sin histórico"

    try:
        df = pd.read_parquet(archivo)
        if df.empty or "fecha" not in df.columns:
            return f"⚠️ {simbolo}: histórico inválido"

        df["fecha"] = pd.to_datetime(df["fecha"])
        df = df[df["fecha"] < pd.Timestamp.today()]
        if df.empty:
            return f"⚠️ {simbolo}: sin datos previos a hoy"

        ultima_fecha = df["fecha"].max()
        resultados = []

        for nombre_est, funcion in desarrollos.items():
            try:
                df_out = funcion(df.copy())
                if df_out is not None and not df_out.empty:
                    df_out["fecha"] = pd.to_datetime(df_out["fecha"])
                    df_out = df_out[df_out["fecha"] == ultima_fecha]
                    if not df_out.empty:
                        df_out["simbolo"] = simbolo
                        df_out["estrategia"] = nombre_est
                        resultados.append(df_out)
            except Exception as e:
                return f"❌ {simbolo} > {nombre_est}: {e}"

        if resultados:
            df_final = pd.concat(resultados)
            df_final = df_final.sort_values("fecha")
            df_final.to_csv(OUTPUT_PATH / f"{simbolo}_senales_diarias.csv", index=False)
        else:
            return f"⏭️ {simbolo}: sin señales"

    except Exception as e:
        return f"❌ {simbolo}: error general: {e}"

# === EJECUCIÓN PARALELA ===
start = time.time()

resultados = Parallel(n_jobs=-1)(
    delayed(procesar_simbolo)(simbolo) for simbolo in tqdm(simbolos, desc="Procesando símbolos")
)

end = time.time()
duracion = end - start

print(f"\n🕒 Tiempo total: {duracion // 60:.0f} min {duracion % 60:.0f} seg")



Borrando señales anteriores en D:\trading\reports\senales_heuristicas\diarias...
✅ Directorio limpio.


Procesando símbolos: 100%|█████| 429/429 [00:23<00:00, 18.50it/s]



🕒 Tiempo total: 0 min 24 seg


In [8]:

# === Ruta de señales heurísticas ===
OUTPUT_PATH = Path("D:/trading/reports/senales_heuristicas/diarias")

# === Cargar todas las señales en un único DataFrame ===
df_lista = []
for archivo in OUTPUT_PATH.glob("*.csv"):
    try:
        df = pd.read_csv(archivo)
        if "simbolo" in df.columns and "signal" in df.columns:
            df_lista.append(df)
    except Exception as e:
        print(f"❌ Error al leer {archivo.name}: {e}")

if not df_lista:
    print("⚠️ No se encontraron señales.")
else:
    df_all = pd.concat(df_lista).dropna(subset=["signal"])
    df_all["signal"] = df_all["signal"].str.lower().str.strip()

    # Clasificación por símbolo
    resumen = df_all.groupby("simbolo")["signal"].apply(lambda s: set(s)).reset_index()
    resumen["buy"] = resumen["signal"].apply(lambda x: "buy" in x)
    resumen["sell"] = resumen["signal"].apply(lambda x: "sell" in x)
    resumen["hold_only"] = resumen["signal"].apply(lambda x: x == {"hold"})

    # Reporte
    print("📈 Símbolos con señal BUY:")
    display(resumen[resumen["buy"]]["simbolo"].tolist())

    print("\n📉 Símbolos con señal SELL:")
    display(resumen[resumen["sell"]]["simbolo"].tolist())

#    print("\n🤝 Símbolos con solo HOLD:")
#    display(resumen[resumen["hold_only"]]["simbolo"].tolist())


📈 Símbolos con señal BUY:


['DG', 'DLTR']


📉 Símbolos con señal SELL:


['DG']

In [9]:
OUTPUT_PATH = Path("D:/trading/reports/senales_heuristicas/diarias")
df_lista = []

# Cargar todas las señales
for archivo in OUTPUT_PATH.glob("*.csv"):
    try:
        df = pd.read_csv(archivo)
        if {"simbolo", "signal", "estrategia"}.issubset(df.columns):
            df_lista.append(df)
    except Exception as e:
        print(f"❌ Error en {archivo.name}: {e}")

if df_lista:
    df_all = pd.concat(df_lista).dropna(subset=["signal"])
    df_all["signal"] = df_all["signal"].str.lower().str.strip()

    # Filtrar solo BUY y SELL
    df_all = df_all[df_all["signal"].isin(["buy", "sell"])]

    # Agrupar estrategias por tipo de señal
    df_pivot = df_all.groupby(["simbolo", "signal"])["estrategia"] \
        .apply(lambda x: ", ".join(sorted(set(x)))).unstack(fill_value="")

    df_pivot.reset_index(inplace=True)
    df_pivot.columns.name = None
    df_pivot.rename(columns={"buy": "Estrategias BUY", "sell": "Estrategias SELL"}, inplace=True)

    display(df_pivot)
else:
    print("⚠️ No se encontraron señales válidas.")

,simbolo,Estrategias BUY,Estrategias SELL
0,DG,bollinger_breakout_v4,gap_open_strategy_v5
1,DLTR,bollinger_breakout_v4,


In [10]:
end1 = time.time()
duracion = end - start1

print(f"\n🕒 Tiempo total: {duracion // 60:.0f} min {duracion % 60:.0f} seg")


🕒 Tiempo total: 0 min 27 seg
